In [1]:
from ipycanvas import Canvas, hold_canvas

In [2]:
from ipywidgets import widgets, HBox, VBox, Layout

In [3]:
import math

In [4]:
robot_canvas_width = 500
robot_canvas_height = 300
    
robot_canvas = Canvas(width=robot_canvas_width, height=robot_canvas_height)

In [66]:
def draw_robot_canvas(canvas, rf):
    """
    Given a canvas to draw on and the points of a robot figure,
    draw a stick representation of the robot and arm.
    """
    with hold_canvas():
        canvas.clear()
        canvas.reset_transform()

        # The origin of the canvas is the top which is upside down so we will
        # move it to the bottom left corner and invert the y-axis
        canvas.translate(0, canvas.height)
        canvas.scale(2.0, -2.0)

        canvas.line_width = 7
        canvas.line_cap = 'round'
        
        # Draw vertical line for robot base
        canvas.begin_path()
        canvas.stroke_style = "black"
        canvas.stroke_lines([rf.b1, rf.b0, rf.j0])
        canvas.stroke()
        
        # Draw first link (j0-j1)
        canvas.begin_path()
        canvas.stroke_style = "green"
        canvas.stroke_lines([rf.j0, rf.j1])
        canvas.stroke()

        # Draw second link (j1-j2)
        canvas.line_width = 5
        canvas.begin_path()
        canvas.stroke_style = "blue"
        canvas.stroke_lines([rf.j1, rf.j2])
        canvas.stroke()
        
        # Draw a line that is 48 inches from the robot base
        canvas.line_width = 1
        canvas.begin_path()
        canvas.stroke_style = "red"
        canvas.stroke_lines([rf.limit1, rf.limit2])
        canvas.stroke()
        


In [67]:
class robot_configuration:
    """
    This class just holds the robot dimensions and the joint state angles
    """
    base_x = 100
    base_height = 5
    base_width = 32
    
    limit = 48
    
    l1 = 32
    j0_degrees = 90
    
    l2 = 32
    j1_degrees = -90

# Create an instance of the class
rc = robot_configuration()

In [68]:
class robot_figure:
    """
    This class provides the task space representation of the key robot
    points for doing range checking and drawing. It depends on the
    robot configuration.
    """
    
    def __init__(self, robot_configuration):
        self.rc = robot_configuration
        self.b0 = [0,0]
        self.b1 = [0,0]
        self.j0 = [0,0]
        self.j1 = [0,0]
        self.j2 = [0,0]
        self.limit1 = [0,0]
        self.limit2 = [0,0]
        self.d2r = math.pi/180 # convert degrees to radians
    
    def update_locations(self):
        #Draw the base first adn the j0 (arm) attachment point
        self.b0 = [self.rc.base_x, 20]
        self.b1 = [self.rc.base_x - self.rc.base_width, 20]

        # Draw a line 48 inches in front of the robot
        self.limit1 = [self.b0[0] + self.rc.limit, self.b0[1]]
        self.limit2 = [self.b0[0] + self.rc.limit, self.b0[1]+200]

        self.j0 = [self.b0[0], self.b0[1]+self.rc.base_height]

        # Add the first link to the base
        self.j1[0] = self.j0[0] + self.rc.l1 * math.cos(self.rc.j0_degrees*self.d2r)
        self.j1[1] = self.j0[1] + self.rc.l1 * math.sin(self.rc.j0_degrees*self.d2r)
        
        # Add the second link to the first link
        # The second joint angle is relative to the first joint angle so they are added
        self.j2[0] = self.j1[0] + self.rc.l2 * math.cos((self.rc.j0_degrees+self.rc.j1_degrees)*self.d2r)
        self.j2[1] = self.j1[1] + self.rc.l2 * math.sin((self.rc.j0_degrees+self.rc.j1_degrees)*self.d2r)


# Create an instance of the class
rf = robot_figure(rc)


In [69]:
# Create Widgets to allow the robot pose to be adjusted in joint space

j0_widget = widgets.FloatSlider(
    value=180,
    min=-180,
    max=180,
    step=0.1,
    description='J0 degrees:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
#    layout=Layout( width='700px' ),
)

def on_value_change_j0(change):
    rc.j0_degrees = (change['new'])
    rf.update_locations()
    draw_robot_canvas(robot_canvas, rf)

j0_widget.observe(on_value_change_x, names='value')

j1_widget = widgets.FloatSlider(
    value=180,
    min=-180,
    max=180.0,
    step=0.1,
    description='J1 Degrees:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
#    layout=Layout( width='700px' ),
)

def on_value_change_j1(change):
    rc.j1_degrees= (change['new'])
    rf.update_locations()
    draw_robot_canvas(robot_canvas, rf)
    
j1_widget.observe(on_value_change_j1, names='value')


In [70]:
VBox([j0_widget, j1_widget])

In [71]:
robot_canvas

Canvas(height=300, width=500)